In [9]:
import numpy as np
import pandas as pd
import os
from PIL import Image
from fastai import *
from fastai.callbacks import CSVLogger, SaveModelCallback

RuntimeError: 
builtin cannot be used as a value:
at C:\Users\Darren\Anaconda3\envs\MacbookPro\lib\site-packages\torchvision\models\detection\_utils.py:14:56
def zeros_like(tensor, dtype):
    # type: (Tensor, int) -> Tensor
    return torch.zeros_like(tensor, dtype=dtype, layout=tensor.layout,
                                                        ~~~~~~~~~~~~~ <--- HERE
                            device=tensor.device, pin_memory=tensor.is_pinned())
'zeros_like' is being compiled since it was called from '__torch__.torchvision.models.detection._utils.BalancedPositiveNegativeSampler.__call__'
at C:\Users\Darren\Anaconda3\envs\MacbookPro\lib\site-packages\torchvision\models\detection\_utils.py:72:12

            # randomly select positive and negative examples
            perm1 = torch.randperm(positive.numel(), device=positive.device)[:num_pos]
            perm2 = torch.randperm(negative.numel(), device=negative.device)[:num_neg]

            pos_idx_per_image = positive[perm1]
     

# Data Visualization

In [2]:
train_dir = '../data/train_set/'
val_dir = '../data/val/'

In [4]:
train_df = pd.read_csv('../data/train_labels.csv')
train_df['path'] = train_df['img_name'].map(lambda x : os.path.join(train_dir, x))
val_df = pd.read_csv('../data/val_labels.csv')
val_df['path'] = val_df['img_name'].map(lambda x : os.path.join(val_dir, x))

In [5]:
df = pd.concat([train_df, val_df], ignore_index=True)
df.head()

,img_name,label,path
0,train_101733.jpg,211,../data/train_set/train_101733.jpg
1,train_101734.jpg,211,../data/train_set/train_101734.jpg
2,train_101735.jpg,211,../data/train_set/train_101735.jpg
3,train_101736.jpg,211,../data/train_set/train_101736.jpg
4,train_101737.jpg,211,../data/train_set/train_101737.jpg


In [6]:
val_idx = [i for i in range(len(train_df), len(df))]

In [7]:
sz = 256
bs = 32

In [8]:
# np.random.seed(42)
# tfms = get_transforms(do_flip=True,flip_vert=True,max_rotate=360,max_warp=0,max_zoom=1.2,max_lighting=0.5,p_lighting=0.5, p_affine=0.5)
src = (ImageList.from_df(df=df,path='./',cols='path') #get dataset from dataset
       .split_by_idx(val_idx)
        .label_from_df(cols='label') #obtain labels from the level column
      )
data= (src.transform(tfms,size=sz,resize_method=ResizeMethod.SQUISH,padding_mode='border') #Data augmentation
        .databunch(bs=bs,num_workers=4) #DataBunch
        .normalize(imagenet_stats) #Normalize
       )

NameError: name 'ImageList' is not defined